I have created a medical chatbot using the Mistral7, with RAG and Llama index. I have prepared a custom dataset using GPT-4 which specializes in mental health care conversations and books that talk about mental health care. The chatbot guides users, provides personalized and emphatetic responses, helping user with their mental health wellness

Install and import Required libraries

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00


To make use of CPU and GPU for inferencing from my LLM

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.12-cp310-cp310-manylinux_2_35_x86_64.whl size=4032366 sha256=7aabdff195942c08826762d2eea8c7bd715968e7e810fafdf97c0731ef250a3a
  Stored in directory: /tmp/pip-ephem-wheel-cache-cd9kfnvu/wheels/07/07/dd/f9549e7045690b8b66dadaa2224a478aa49a61edc8d368a62e
Successfully built llama-cpp-python


Llama index library supports retrieval based augmentation for LLMs

Can create embeddings for the text in PDF. So when I query something, I am able to get answer for my query from the mistral model

In [ ]:
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.0/808.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [ ]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [ ]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
!pip -q install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("I feel the urge to overeat everyday, what to do?")

In [ ]:
print(response)

 It's understandable that you may feel the urge to overeat every day. One approach could be to identify any patterns in your eating habits, such as specific times when you eat more or less, or certain emotions that trigger changes in your appetite. Addressing the root cause of the stress and finding healthier coping mechanisms can contribute to stabilizing your eating habits. Additionally, starting with small, manageable steps, such as adding one serving of vegetables to one meal a day or choosing a piece of fruit as a snack, can make the process more achievable. Planning meals in advance and having healthy options readily available can also help reduce overwhelm and make healthier choices easier. It's important to remember that making changes can feel overwhelming, but taking small steps consistently can lead to long-term improvements in your nutritional intake and overall well-being.


Inorder to end the conversation I clicked on stop running. This is not an error

In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

I have been having difficulties with sleeping every night. Could you help me?


Llama.generate: prefix-match hit


 Yes, I can definitely help you with your sleep difficulties. It sounds like you may be experiencing some common issues that can impact sleep quality, such as stress and difficulty relaxing before bed. One helpful strategy to try is creating a calming bedtime routine that includes activities like reading, gentle stretching, or listening to calming music. Additionally, limiting screen time before bed can also help improve sleep quality. Have you tried any relaxation techniques before bed, such as deep breathing exercises? If not, it might be worth giving them a try. Remember, creating a consistent sleep routine can contribute to better sleep over time.
I try to go to bed at the same time everyday, but inspite of trying hard to close my eyes and lay down, I cant sleep. I keep getting some thoughts


Llama.generate: prefix-match hit


 It sounds like you are having difficulty falling asleep despite trying to establish a consistent sleep routine. There could be several reasons for this, such as stress or anxiety, physical discomfort, or an irregular sleep schedule. One strategy that may help is to engage in relaxation techniques before bedtime, such as deep breathing exercises, progressive muscle relaxation, or meditation. It's also important to create a comfortable sleep environment by keeping your bedroom cool, dark, and quiet, and investing in a supportive mattress and pillows. If you continue to have difficulty sleeping despite making these changes, it may be helpful to consult with a healthcare professional who can provide further guidance and support.
Is there a specific phone number or website that I can check inorder to check my issue


...

Llama.generate: prefix-match hit


 You're welcome! Remember that taking care of your mental health is a vital part of your overall well-being. If you need any further support or guidance in the future, don't hesitate to reach out.
exit


Llama.generate: prefix-match hit


 I understand that you would like to exit this conversation. If you or someone you know is experiencing thoughts of suicide, it is crucial to contact emergency services, a crisis hotline, or a trusted individual who can assist in connecting with the appropriate support. Your safety is the top priority. Is there anything else I can help you with?


KeyboardInterrupt: ignored